In [1]:
base_dir = '/data/troubleshooting_dti_failure/'
%cd {base_dir}

/data/troubleshooting_dti_failure


## Dcm2niix

In [2]:
mkdir dcm2niix_output

In [3]:
tar_list = !tar xvf dti_test_dicoms_001_001.tar.gz
tar_list[:5]

['dti_dicoms/',
 'dti_dicoms/mr_0017/',
 'dti_dicoms/mr_0017/edti_2mm_cdif99-00073.dcm',
 'dti_dicoms/mr_0017/edti_2mm_cdif99-00163.dcm',
 'dti_dicoms/mr_0017/edti_2mm_cdif99-00087.dcm']

In [4]:
ls

dcm2niix_output/  dti_dicoms/  dti_test_dicoms_001_001.tar.gz*


In [5]:
!dcm2niix -b y -z i -x n -t n -m n -f dwi -o /data/troubleshooting_dti_failure/dcm2niix_output \
-s n -v n /data/troubleshooting_dti_failure/dti_dicoms/mr_0013/edti_2mm_cdif09_g01-00001.dcm

Compression will be faster with /usr/local/bin/pigz
Chris Rorden's dcm2niiX version v1.0.20170314 (64-bit Linux)
Found 1680 DICOM image(s)
Reorienting edti_2mm_cdif09_G01 : 20 GE DTI vectors: please validate. isCol=1 sliceDir=3 flp=0 0 1
Convert 1680 DICOM as /data/troubleshooting_dti_failure/dcm2niix_output/dwi (128x128x84x20)
Conversion required 2.709230 seconds.


In [6]:
ls dcm2niix_output/

dwi.bval  dwi.bvec  dwi.json  dwi.nii.gz


## Nipype 

In [7]:
from nipype import __version__
__version__

u'0.13.0-dev'

In [8]:
from nipype import Function, Node
from nipype.interfaces.base import isdefined
from nipype.interfaces.dcm2nii import Dcm2nii, Dcm2niix
import os


In [9]:
%cd {base_dir}
tmpdir = '/data/troubleshooting_dti_failure/nipype_dicoms'
%rm -rf {tmpdir}

%mkdir {tmpdir}
!cp -R dti_dicoms/mr_0013/* {tmpdir}
%cd {tmpdir}

/data/troubleshooting_dti_failure
/data/troubleshooting_dti_failure/nipype_dicoms


In [10]:
# need to be abspaths!
dicoms = list(map(os.path.abspath, os.listdir('.')))
dicoms[:5]

['/data/troubleshooting_dti_failure/nipype_dicoms/edti_2mm_cdif09_g01-01440.dcm',
 '/data/troubleshooting_dti_failure/nipype_dicoms/edti_2mm_cdif09_g01-00974.dcm',
 '/data/troubleshooting_dti_failure/nipype_dicoms/edti_2mm_cdif09_g01-00793.dcm',
 '/data/troubleshooting_dti_failure/nipype_dicoms/edti_2mm_cdif09_g01-01367.dcm',
 '/data/troubleshooting_dti_failure/nipype_dicoms/edti_2mm_cdif09_g01-00909.dcm']

In [11]:
converter = 'dcm2niix'
prefix_name = 'dwi'
convertnode = Node(Dcm2niix(), name='convert')
convertnode.base_dir = os.path.abspath(tmpdir)
convertnode.inputs.source_names = dicoms
convertnode.inputs.out_filename = 'dwi'
convertnode.inputs.terminal_output = 'allatonce'

In [12]:
res = convertnode.run()

170705-20:08:48,121 workflow INFO:
	 Executing node convert in dir: /data/troubleshooting_dti_failure/nipype_dicoms/convert
170705-20:08:48,853 workflow INFO:
	 Running: dcm2niix -b y -z i -x n -t n -m n -f dwi -o /data/troubleshooting_dti_failure/nipype_dicoms/convert -s n -v n /data/troubleshooting_dti_failure/nipype_dicoms/convert/edti_2mm_cdif09_g01-01440.dcm


In [13]:
res.outputs


bids = /data/troubleshooting_dti_failure/nipype_dicoms/convert/dwi.json
bvals = <undefined>
bvecs = <undefined>
converted_files = /data/troubleshooting_dti_failure/nipype_dicoms/convert/dwi.nii.gz